<a href="https://colab.research.google.com/github/sunnysavita10/Indepth-GENAI/blob/main/RAG_With_Knowledge_graph(Neo4j).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# langchain-core

contains simple, core abstractions that have emerged as a standard, as well as LangChain Expression Language as a way to compose these components together. This package is now at version 0.1 and all breaking changes will be accompanied by a minor version bump.

# langchain-community
contains all third party integrations. We will work with partners on splitting key integrations out into standalone packages over the next month.

# langchain
contains higher-level and use-case specific chains, agents, and retrieval algorithms that are at the core of your application's cognitive architecture. We are targeting a launch of a stable 0.1 release for langchain in early January.#

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.1/208.1 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.6/296.6 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 32.9 MB/s eta 0:00:00
   ━━

In [2]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [ ]:
from google.colab import userdata
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')

In [4]:
from typing import Tuple, List, Optional

In [5]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [6]:
from langchain_core.runnables import ConfigurableField

In [7]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase


In [8]:
import os

In [9]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [10]:
from langchain_community.vectorstores import Neo4jVector

In [12]:
NEO4J_URI="neo4j+s://db131b59.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="XaGKlKWY0QaJPnhQirft1alfp8m0hGCUnd11hLquBKQ"

In [13]:
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [14]:
from langchain_community.graphs import Neo4jGraph

In [15]:
graph = Neo4jGraph()

In [17]:
!pip install langchain-core

In [22]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 18.2 MB/s eta 0:00:00


In [19]:
!pip install -qU langchain-text-splitters

In [34]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_core.documents import Document
import re
from typing import List

def extract_urls(text: str) -> List[str]:
    """Extract URLs from the provided text using regex."""
    url_pattern = r'https?://[^\s]+'
    urls = re.findall(url_pattern, text)
    return urls

def load_pdfs_from_directory(directory_path: str) -> List[Document]:
    """Load PDFs from the specified directory and extract URLs."""
    loader = DirectoryLoader(directory_path, glob="**/*.pdf", loader_cls=PyPDFLoader)
    pdf_docs = loader.load()

    all_urls = []  # List to hold all extracted URLs
    documents = []  # List to hold Document objects

    for pdf_doc in pdf_docs:
        # Extract text from the document
        text = pdf_doc.page_content  # Adjust this based on how your PDF content is accessed

        # Extract URLs from the text
        urls = extract_urls(text)
        all_urls.extend(urls)

        # Create a Document object and append it to the documents list
        documents.append(Document(page_content=text, metadata={"source": pdf_doc.metadata.get("source")}))

    # Print extracted URLs
    print("Extracted URLs:")
    for url in all_urls:
        print(url)

    return documents  # Return the list of Document objects


In [41]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_core.documents import Document
import re
from typing import List

def extract_urls(text: str) -> List[str]:
    """Extract URLs from the provided text using regex."""
    url_pattern = r'https?://[^\s]+'
    urls = re.findall(url_pattern, text)
    return urls

def load_pdfs_from_directory(directory_path: str) -> List[Document]:
    """Load PDFs from the specified directory and extract URLs."""
    loader = DirectoryLoader(directory_path, glob="**/*.pdf", loader_cls=PyPDFLoader)
    pdf_docs = loader.load()

    all_urls = []  # List to hold all extracted URLs
    documents = []  # List to hold Document objects

    for pdf_doc in pdf_docs:
        # Extract text from the document
        text = pdf_doc.page_content  # Adjust this based on how your PDF content is accessed

        # Extract URLs from the text
        urls = extract_urls(text)
        all_urls.extend(urls)

        # Create a Document object and append it to the documents list
        documents.append(Document(page_content=text, metadata={"source": pdf_doc.metadata.get("source")}))

    # Print extracted URLs
    print("Extracted URLs:")
    for url in all_urls:
        print(url)

    print(f"Loaded {len(documents)} documents.")  # Print the number of documents loaded
    print(f"Type of first document: {type(documents[0])}")  # Check the type of the first document

    return documents  # Return the list of Document objects

# Usage
from langchain.text_splitter import TokenTextSplitter

# Load PDFs
documents = load_pdfs_from_directory("/content/data")

# Initialize the text splitter
text_splitter = TokenTextSplitter(chunk_size=400, chunk_overlap=40)

# Check the type before splitting
print(f"Type of documents: {type(documents)}")
if documents:
    print(f"Type of first document: {type(documents[0])}")

# Now split the documents
split_documents = text_splitter.split_documents(documents)


Extracted URLs:
https://api.whatsapp.com/send/?phone=9105575000&text=I+want+to+order+Sunn+Galaa+Aa
https://play.google.com/store/apps/details?id=com.bo
https://apps.apple.com/in/app/sunn
https://api.whatsapp.com/send/?phone=9105575000&text=I+want+to+order+Sukhmani+Sehaj+
https://api.whatsapp.com/send/?phone=9105575000&text=I+want+to+order+Sukhmani+Sehaj+
https://play.google.com/store/apps/details?id=com.books.sukhmani_sehaj_gobind_gunn
https://apps.apple.com/mx/app/sukhmani
https://play.google.com/store/apps/details?id=com.books.sukhmani_sehaj_gobind_gunn
https://ap
https://eternalpathbooks.com/sikhism
https://www.instagram.com/eternalpathbooks/
https://www.facebook.com/EternalPathBooks
https://eternalpathbooks.com
Loaded 5 documents.
Type of first document: <class 'langchain_core.documents.base.Document'>
Type of documents: <class 'list'>
Type of first document: <class 'langchain_core.documents.base.Document'>


In [ ]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0125")

In [26]:
GROQ_API_KEY="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"

In [27]:
import getpass
import os

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")

Enter your Groq API key: ··········


In [29]:
!pip install langchain_groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 11.0 MB/s eta 0:00:00


In [30]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [31]:
from langchain_experimental.graph_transformers import LLMGraphTransformer


# Create the LLMGraphTransformer with groqllm
llm_transformer = LLMGraphTransformer(llm=llm)



In [42]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [43]:
graph_documents

[GraphDocument(nodes=[Node(id='Eternalpath Books', type='Organization', properties={}), Node(id='Swami Balbir Singh Ji', type='Person', properties={}), Node(id='Guru Nanak Dev Ji', type='Person', properties={})], relationships=[Relationship(source=Node(id='Eternalpath Books', type='Organization', properties={}), target=Node(id='Swami Balbir Singh Ji', type='Person', properties={}), type='AUTHOR', properties={}), Relationship(source=Node(id='Eternalpath Books', type='Organization', properties={}), target=Node(id='Guru Nanak Dev Ji', type='Person', properties={}), type='INFLUENCED_BY', properties={}), Relationship(source=Node(id='Swami Balbir Singh Ji', type='Person', properties={}), target=Node(id='Japji Sahib', type='Scripture', properties={}), type='COMMENTATOR', properties={}), Relationship(source=Node(id='Japji Sahib', type='Scripture', properties={}), target=Node(id='Guru Nanak Dev Ji', type='Person', properties={}), type='WRITTEN_BY', properties={})], source=Document(metadata={'so

In [44]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [45]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [46]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [47]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [48]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [49]:
showGraph()

GraphWidget(layout=Layout(height='640px', width='100%'))

GraphWidget(layout=Layout(height='640px', width='100%'))

In [50]:
from typing import Tuple, List, Optional

In [51]:
from langchain_community.vectorstores import Neo4jVector

In [52]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 16.6 MB/s eta 0:00:00


In [55]:
!pip install langchain-huggingface

In [56]:
from langchain_huggingface import HuggingFaceEmbeddings

In [57]:
embeddings_hf = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [58]:
vector_index = Neo4jVector.from_existing_graph(
    embeddings_hf,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [59]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [60]:
from langchain_core.pydantic_v1 import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [61]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [62]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a concise and polite customer service bot. ",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [63]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [64]:
entity_chain.invoke({"question": "Author of Sunn Gaala Aakaas kee"}).names

['Sunn Gaala Aakaas kee']

In [65]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [66]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


In [67]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [68]:
print(structured_retriever("Sunn Gaala Aakaas kee"))

Sukhmani Sehaj Gobind Gunn Naam - WRITTEN_BY -> Swami Balbir Singh Ji
Sukhmani Sehaj Gobind Gunn Naam - BASED_ON -> Guru Arjan Dev Ji
Sukhmani Sehaj Gobind Gunn Naam - RELATED_WORK -> Amrit Myi Baniya Da Guldasta
Sukhmani Sehaj Gobind Gunn Naam - RELATED_WORK -> Kaho Kabeer Hum Aaese JanyaSukhmani Sehaj Gobind Gunn Naam - BASED_ON -> Guru Arjan Dev Ji
Eternalpath Books - INFLUENCED_BY -> Guru Nanak Dev Ji
Japji Sahib - WRITTEN_BY -> Guru Nanak Dev Ji


In [69]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [70]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [71]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [72]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [74]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatGroq(model="llama3-8b-8192",temperature=0,)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [75]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [76]:
prompt = ChatPromptTemplate.from_template(template)

In [77]:
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [99]:
chain.invoke({"question": "instagram profile "})

Search query: instagram profile 


"Eternalpath Books' Instagram profile is [https://www.instagram.com/eternalpathbooks/](https://www.instagram.com/eternalpathbooks/)."